In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import os
import h5py
import math
import pickle

cwd = os.getcwd()

In [9]:
h_dev_path = os.path.join(cwd, '/VSD_2014_December_official_release/Hollywood-dev/')
h_test_path = os.path.join(cwd,'/VSD_2014_December_official_release/Hollywood-test/')
h_generalized_path = os.path.join(cwd,'/VSD_2014_December_official_release/YouTube-gen/')
save_path = os.path.join(cwd,'/VSD_2014_December_official_release/data_arrays/')

visual_features = ['CM', 'CNH', 'HOG', 'LBP']
auditory_features = ['AE', 'BER', 'BW', 'MFCC', 'RMS', 'SC', 'SF', 'ZCR']
# the only concept with interesting labels is blood : {'high', 'low', 'medium', 'unnoticeable'}
visual_concepts = ['blood', 'carchase', 'coldarms', 'fights', 'fire', 'firearms', 'gore'] 
auditory_concepts = ['explosions', 'gunshots', 'screams'] # also contain 'nothing' label

In [ ]:
def get_all_annotations(dev_or_test_or_generalized, path = save_path):  
    f = open(save_path + dev_or_test_or_generalized + r'/annotations/all_annotations.p', 'rb')
    return pickle.load(f)

def get_annotations(dev_or_test_or_generalized, audio_or_visual, path = save_path):
    f = open(save_path + dev_or_test_or_generalized + r'/annotations/' +  'Y_' + audio_or_visual + '_dictionary.p', 'rb')
    return pickle.load(f)   

def get_annotation(dev_or_test_or_generalized, audio_or_visual, annotation, path = save_path):
    f = open(save_path + dev_or_test_or_generalized + r'/annotations/' +  'Y_' + audio_or_visual + '_dictionary.p', 'rb')
    d = pickle.load(f)
    return np.array(d[annotation])

def get_data(dev_or_test_or_generalized, save_path = save_path, prefix = '', extension = ''):
    f = open(save_path + dev_or_test_or_generalized + r'/' + prefix + 'data' + extension + '.p', 'rb')
    return pickle.load(f)   
    
def get_feature(dev_or_test_or_generalized, feature_name, path = save_path):
    if feature_name in visual_features:
        f = open(save_path + dev_or_test_or_generalized + r'/visual_features/' + feature_name + '.p', 'rb')
        return pickle.load(f)
    elif feature_name in auditory_features:
        f = open(save_path + dev_or_test_or_generalized + r'/audio_features/' + feature_name + '.p', 'rb')
        return pickle.load(f)    
    
def get_intervals(dev_or_test_or_generalized, path = save_path):
    f = open(save_path + dev_or_test_or_generalized + r'/annotations/' +  'intervals_dictionary.p', 'rb')
    return pickle.load(f)    

def save_all_annotations(dev_or_test_or_generalized, save_path = save_path): 
    all_annotations = []
    annotations_visual = get_annotations(dev_or_test_or_generalized, 'visual')
    for concept in annotations_visual:
        all_annotations.append(annotations_visual[concept])
    annotations_audio = get_annotations(dev_or_test_or_generalized, 'audio')
    for concept in annotations_audio:
        all_annotations.append(annotations_audio[concept])
    all_annotations = list(zip(*all_annotations))
    f = open(save_path + dev_or_test_or_generalized + r'/annotations/all_annotations.p', 'wb')
    pickle.dump(all_annotations, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()   
    
def save_annotations(path, save_path):
    os.chdir(path)
    Y_visual = {}
    Y_audio = {}
    intervals = {}
    for vis_con in visual_concepts + ['violence']:
        Y_visual[vis_con] = []
        intervals[vis_con] = []
    for audio_con in auditory_concepts: # 25 frames/second
        Y_audio[audio_con] = []
        intervals[audio_con] = []
    l = 0
    i = 0
        
    for file in os.listdir(path + r'features'):
        if 'visual' in file:
            movie_name = file[0: file.rfind('_')]  
            f1 = h5py.File(path + r'features/' + movie_name + '_visual.mat')
            f2 = h5py.File(path + r'features/' + movie_name + '_auditory.mat')
            l = min([f1['CM'].shape[0], f2['ZCR'].shape[0]]) # cut to have same indexing  
            #print('From ' + str(i) + ' to ' + str(i + l - 1) + ' are features of ' + movie_name)
            f1.close()
            f2.close()
            
            for vis_con in visual_concepts + ['violence']:
                if 'generalized' in save_path and vis_con == 'violence':
                    annotation_file_path = path + r'annotations/' + movie_name + '.txt'
                else:
                    annotation_file_path = path + r'annotations/' + movie_name + '_' + vis_con + '.txt'
                if os.path.isfile(annotation_file_path):
                    annotation_file = open(annotation_file_path, 'r')
                    if vis_con == 'blood' :
                        Y_visual[vis_con].extend(['0'] * l)
                        for line in annotation_file.readlines():
                            contents = line.split(' ') 
                            if len(contents) >= 3:
                                a, b, c = line.split(' ')[0:3]
                                c = c.replace('\n', '')
                                a = int(a)
                                b = min([int(b), l - 1]) # frame intervals are closed, indexing starts from 0
                                for j in range(a, b + 1):
                                    Y_visual[vis_con][i + j] = c
                                intervals[vis_con].append((i + a, i + b, c))
                    else:        
                        Y_visual[vis_con].extend([0] * l)
                        for line in annotation_file.readlines():
                            contents = line.split(' ') 
                            if len(contents) >= 2:
                                a, b = line.split(' ')[0:2]
                                a = int(a)
                                b = min([int(b), l - 1])
                                for j in range(a, b + 1):
                                    Y_visual[vis_con][i + j] = 1
                                intervals[vis_con].append((i + a, i + b))
                    annotation_file.close()
                else:
                    Y_visual[vis_con].extend([0] * l)
                    
            for audio_con in auditory_concepts:
                annotation_file_path = path + r'annotations/' + movie_name + '_' + audio_con + '.txt'
                if os.path.isfile(annotation_file_path):
                    annotation_file = open(annotation_file_path, 'r')
                    Y_audio[audio_con].extend([0] * l)
                    for line in annotation_file.readlines():
                        contents = line.split(' ')
                        if len(contents) >= 2:
                            a, b = line.split(' ')[0:2]
                            a = math.floor(float(a) * 25) # 25 frames per second
                            b = min([math.ceil(float(b) * 25), l - 1])
                            if not (len(contents) >= 3 and 'nothing' in contents[2]):
                                for j in range(a, b + 1):
                                    Y_audio[audio_con][i + j] = 1
                                intervals[audio_con].append((i + a, i + b))
                    annotation_file.close()                
                else:
                    Y_audio[audio_con].extend([0] * l)
            i += l
            
    f = open(save_path + r'annotations/' + 'Y_visual_dictionary.p', 'wb')
    pickle.dump(Y_visual, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
    f = open(save_path + r'annotations/' + 'Y_audio_dictionary.p', 'wb')
    pickle.dump(Y_audio, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
    f = open(save_path + r'annotations/' + 'intervals_dictionary.p', 'wb')
    pickle.dump(intervals, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()
    print(intervals)
    
def save_data(dev_or_test_or_generalized, save_path = save_path):
    data = []
    features = []
    for feature_name in visual_features + auditory_features:
        features.append(get_feature(dev_or_test_or_generalized, feature_name)) 
    data = np.concatenate(features, axis = 1)
    f = open(save_path + dev_or_test_or_generalized + r'/data.p', 'wb')
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()   
    
def save_data_compressed(dev_or_test_or_generalized, window, save_path = save_path, prefix = '', extension = ''):
    data = get_data(dev_or_test_or_generalized, save_path, prefix, extension)
    data_compressed = []
    for i in range(0, data.shape[0] - window, window):
        data_compressed.append(np.concatenate(data[i: i + window]))
    i += window
    if i < data.shape[0]:
        if window - data.shape[0] + i == 0:
            data_compressed.append(np.concatenate(data[i: data.shape[0]]))
        else:
            data_compressed.append(np.concatenate(np.vstack((data[i: data.shape[0]], [data[-1]] * 
                                                             (window - data.shape[0] + i)))))
    data_compressed = np.array(data_compressed)
    f = open(save_path + dev_or_test_or_generalized + r'/' + prefix + 'data' + extension + '_compressed_' + 
             str(window) + '.p', 'wb')
    pickle.dump(data_compressed, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()             
    
def save_data_pca(dev_or_test_or_generalized, n_components, save_path = save_path):
    data = get_data(dev_or_test_or_generalized)
    pca = PCA(n_components = n_components)    
    f = open(save_path + dev_or_test_or_generalized+ r'/data_pca_' + str(n_components) + '.p', 'wb')
    pickle.dump(pca.fit_transform(data), f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()    
    
def save_data_timestamped(dev_or_test_or_generalized, timestamp, save_path = save_path): 
    data = get_data(dev_or_test_or_generalized, save_path = save_path)
    l = data.shape[0]
    timestamped_data = [] # timestamps = [timestamp] * (l // timestamp) + [l - timestamp * (l // timestamp)]
    i = 0
    while i < l:
        timestamped_vector = []
        for i in range(i, min((i + timestamp, l))):
            timestamped_vector.append(data[i]) 
        timestamped_data.append(timestamped_vector)
        i += 1
    timestamped_data = np.array(timestamped_data)
    f = open(save_path + dev_or_test_or_generalized+ r'/data_timestamped_' + str(timestamp) + '.p', 'wb')
    pickle.dump(timestamped_data, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()  
    
def save_feature(path, save_path, feature_name):
    os.chdir(path)
    feature = []
    i = 0
    l = 0
    
    for file in os.listdir(path + r'features'):
        if 'visual' in file:
            movie_name = file[0: file.rfind('_')]           
            f1 = h5py.File(path + r'features/' + movie_name + '_visual.mat')
            f2 = h5py.File(path + r'features/' + movie_name + '_auditory.mat')
            l = min([f1['CM'].shape[0], f2['ZCR'].shape[0]]) # cut to have same indexing 
            
            if feature_name in visual_features:
                for j in range(l):
                    feature.append(np.nan_to_num(f1[feature_name][j]))
            elif feature_name in auditory_features:
                for j in range(l):
                    feature.append(np.nan_to_num(f2[feature_name][j]))
            f1.close()
            f2.close()
    
    if feature_name in visual_features:    
        f = open(save_path + r'visual_features/' + feature_name + '.p', 'wb')
        pickle.dump(feature, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()
    elif feature_name in auditory_features:    
        f = open(save_path + r'audio_features/' + feature_name + '.p', 'wb')
        pickle.dump(feature, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

In [ ]:
def print_stats(path = h_dev_path + r'annotations/'):
    violence_intervals = {}
    total_movies_violence_frames = {}
    for file in os.listdir(path):
        if 'violence' in file:
            movie = file[0: file.rfind('_')]
            violence_intervals[movie] = get_intervals_no_labels(path + file, 'violence')
            total_movies_violence_frames[movie] = get_total_frames(violence_intervals[movie])
    print_concepts_stats(path, violence_intervals, total_movies_violence_frames)
    
def get_intervals_no_labels(file_path, concept):
    f = open(file_path, 'r')
    intervals = []
    is_audio = concept in auditory_concepts
    for line in f.readlines():
        contents = line.split(' ')
        if len(contents) >= 2:
            a, b = contents[0: 2]
            if is_audio: # need to convert to frame intervals
                a = math.floor(float(a) * 25) 
                b = math.ceil(float(b) * 25)
                if not (len(contents) >= 3 and 'nothing' in contents[2]): 
                    intervals.append((a, b))
            else:
                intervals.append((int(a), int(b)))
    f.close()
    return intervals 

def get_intervals_with_labels(file_path, concept):
    f = open(file_path, 'r')
    intervals = []
    is_audio = concept in auditory_concepts
    for line in f.readlines():
        contents = line.split(' ') 
        if len(contents) >= 3:
            a, b, c = contents[0: 3]
            if is_audio:
                a = math.floor(float(a) * 25) 
                b = math.ceil(float(b) * 25)
                intervals.append((a, b, c.replace('\n', '')))                
            else:
                intervals.append((int(a), int(b), c.replace('\n', '')))
    f.close()
    return intervals
    
def print_concepts_stats(path, violence_intervals, total_movies_violence_frames):
    concept_movies = {}
    for concept in visual_concepts + auditory_concepts:
        concept_movies[concept] = []        
    for file in os.listdir(path):
        for concept in visual_concepts + auditory_concepts:
            if concept in file:
                concept_movies[concept].append(file[0 : file.rfind('_')])
            
    for concept in visual_concepts[1 :] + auditory_concepts: # blood special case, has interesting labels
        print(concept)
        total_violent_frames = 0
        concept_frames = 0
        concept_violence_frames = 0
        for movie in concept_movies[concept]:
            intervals = get_intervals_no_labels(path + movie + '_' + concept + '.txt', concept)
            total_frames = get_total_frames(intervals)
            overlap = get_overlap_no_labels(intervals, violence_intervals[movie])
            total_violent_frames += total_movies_violence_frames[movie]
            concept_frames += total_frames
            concept_violence_frames += overlap
            print('In ' + movie + ' there are ' + str(total_movies_violence_frames[movie]) + ' frames with violence')
            print('In ' + movie + ' there are ' + str(total_frames) + ' frames with ' + concept)
            print('In ' + movie + ' ' + str(overlap) + ' ' + concept + ' frames are also frames with violence')
            if total_frames > 0:
                print('In ' + movie + ' ' + str(overlap / total_frames * 100) + '% of frames with ' + concept + 
                      ' are frames with violence')
            if total_movies_violence_frames[movie] > 0:
                print('In ' + movie + ' ' + str(overlap / total_movies_violence_frames[movie] * 100) + 
                      '% of frames with violence contain ' + concept)            
        print()
        print('In total, out of ' + str(total_violent_frames) + ' frames with violence, ' + 
              str(concept_violence_frames) + ' contain ' + concept)
        if total_violent_frames > 0:
            print('That is ' + str(concept_violence_frames / total_violent_frames * 100) +'%')
        print('In total, out of ' + str(concept_frames) + ' frames with ' + concept + ' ' + 
              str(concept_violence_frames) + ' contain violence')
        if concept_frames > 0:
            print('That is ' + str(concept_violence_frames / concept_frames * 100) +'%')
        print()
        print()
        
    print('blood')
    total_violent_frames = 0
    blood_frames = 0
    blood_labelled_frames = {}
    blood_violence_frames = 0
    blood_labelled_violence_frames = {}
    for movie in concept_movies['blood']:
        total_frames = 0
        movie_overlap = 0
        total_violent_frames += total_movies_violence_frames[movie]
        intervals_with_labels = get_intervals_with_labels(path + movie + '_blood.txt', 'blood')
        labelled_intervals = get_labelled_intervals(intervals_with_labels)
        for label in labelled_intervals:
            overlap = get_overlap_no_labels(labelled_intervals[label], violence_intervals[movie])
            movie_overlap += overlap
            blood_violence_frames += overlap
            labelled_frames = get_total_frames(labelled_intervals[label])
            total_frames += labelled_frames
            blood_frames += labelled_frames
            if label not in blood_labelled_frames:
                blood_labelled_frames[label] = labelled_frames
            else:
                blood_labelled_frames[label] += labelled_frames
            if label not in blood_labelled_violence_frames:
                blood_labelled_violence_frames[label] = overlap
            else:
                blood_labelled_violence_frames[label] += overlap
            print('In ' + movie + ' there are ' + str(total_movies_violence_frames[movie]) + ' frames with violence')
            print('In ' + movie + ' there are ' + str(labelled_frames) + ' frames with blood labelled ' + label)
            print('In ' + movie + ' ' + str(overlap) + ' frames with blood labelled ' + label + 
                  ' frames are also frames with violence')
            if labelled_frames > 0:
                print('In ' + movie + ' ' + str(overlap / labelled_frames * 100) + 
                      '% of frames with blood labellled ' + label + ' are frames with violence')
            if total_movies_violence_frames[movie] > 0:
                print('In ' + movie + ' ' + str(overlap / total_movies_violence_frames[movie] * 100) + 
                      '% of frames with violence contain blood labelled ' + label)
        print()
        print('In ' + movie + ' there are ' + str(total_frames) + ' frames with blood')
        print('In ' + movie + ' ' + str(movie_overlap) +  ' blood frames are also frames with violence')
        if total_frames > 0:
            print('That is ' + str(movie_overlap / total_frames * 100) +'%')
        if total_movies_violence_frames[movie] > 0:
            print('In ' + movie + ' ' + str(movie_overlap / total_movies_violence_frames[movie] * 100) + 
                  '% of frames with violence contain ' + 'blood')            
        print()
    print()
    for label in blood_labelled_frames:
        print('In total, out of ' + str(total_violent_frames) + ' frames with violence, ' + 
              str(blood_labelled_frames[label]) + ' contain blood labelled ' + label)
        if total_violent_frames > 0:
            print('That is ' + str(blood_labelled_frames[label] / total_violent_frames * 100) + '%')
        print('In total, out of ' + str(blood_labelled_frames[label]) + ' frames with ' + label + ' blood, ' + 
              str(blood_labelled_violence_frames[label]) + ' are also frames wih violence')
        if blood_labelled_frames[label] > 0:
            print('That is ' + str(blood_labelled_violence_frames[label] / blood_labelled_frames[label] * 100) + '%')
    print()
    print('In total, out of ' + str(total_violent_frames) + ' frames with violence, ' + 
          str(blood_violence_frames) + ' contain blood')
    if total_violent_frames > 0:
        print('That is ' + str(blood_violence_frames / total_violent_frames * 100) +'%')
    print('In total, out of ' + str(blood_frames) + ' frames with ' + 'blood' + ' ' + 
          str(blood_violence_frames) + ' contain violence')
    if blood_frames > 0:
        print('That is ' + str(blood_violence_frames / blood_frames * 100) +'%')
    print()
    print()
            
            
def get_labelled_intervals(intervals_with_labels):
    labelled_intervals = {}
    labels = get_labels(intervals_with_labels)
    for label in labels:
        labelled_intervals[label] = []
    for interval_with_label in intervals_with_labels:
        labelled_intervals[interval_with_label[2]].append(interval_with_label[0: 2])
    return labelled_intervals     
        
def get_labels(intervals_with_labels):
    labels = set()
    for i in range(len(intervals_with_labels)):
        labels.add(intervals_with_labels[i][2])
    return labels
            
def get_overlap_no_labels(intervals1, intervals2): 
    i = 0
    j = 0
    frames = 0 
    while i < len(intervals1) and j < len(intervals2):
        a, b = intervals1[i]
        x, y = intervals2[j]
        if a >= x and b <= y:
            frames += b - a + 1
            i += 1
        elif x >= a and y <= b:
            frames += y - x + 1
            j += 1
        elif a <= x and x <= b and b <= y:
            frames += b - x + 1
            i += 1
        elif a >= x and a <= y and y <= b:
            frames += y - a + 1
            j += 1
        elif b <= x:
            i += 1
        elif a >= y:
            j += 1
    return frames

def get_total_frames(intervals):
    frames = 0
    for i in range(len(intervals)):
        frames += intervals[i][1] - intervals[i][0] + 1
    return frames                       

In [ ]:
def map2014(predicted_intervals, intervals):
    score = 0
    hits = 0
    m = len(predicted_intervals)
    found_intervals = set()
    j = 0
    for i in range(m):
        flag, hit_interval = is_hit(predicted_intervals[i], intervals) 
        if flag: # non-new hits do not count as false positives
            if hit_interval not in found_intervals:
                hits += 1
                found_intervals.add(hit_interval)
                j += 1
                score += hits / j
        else:
            j += 1
    return score / len(intervals)

def map_classic(predicted_intervals, intervals): # multiple hits on the same interval counted
    score = 0
    hits = 0
    m = len(predicted_intervals)
    j = 0
    for i in range(m):
        flag, hit_interval = is_hit(predicted_intervals[i], intervals) 
        if flag: 
            hits += 1
            j += 1
            score += hits / j
        else:
            j += 1
    return score / len(intervals)

def is_hit(interval, intervals):
    flag = False
    hit_interval = (0, 0)
    for i in range(len(intervals)):
        overlap = get_overlap(interval, intervals[i])
        if overlap / (intervals[i][1] - intervals[i][0]) > 0.5:
            flag = True
            hit_interval = intervals[i]
    return flag, hit_interval

def get_overlap(interval1, interval2):
    a, b = interval1
    x, y = interval2
    if a >= x and b <= y:
        return b - a + 1
    elif x >= a and y <= b:
        return y - x + 1
    elif a <= x and x <= b and b <= y:
        return b - x + 1
    elif a >= x and a <= y and y <= b:
        return y - a + 1
    return 0

def print_conf_matrix(predicted_frames, frames):
    tn, fp, fn, tp = 0, 0, 0, 0
    l = len(frames)
    for i in range(l):
        if predicted_frames[i] == 0 and frames[i] == 0:
            tn+= 1
        elif predicted_frames[i] == 1 and frames[i] == 0:
            fp+= 1
        elif predicted_frames[i] == 0 and frames[i] == 1:
            fn+= 1
        elif predicted_frames[i] == 1 and frames[i] == 1:
            tp+= 1
    precision, recall = 0, 0            
    if tp + fp > 0:
        precision = tp / (tp + fp)
    if tp + fn > 0:
        recall = tp / (tp + fn)
    print('Precision: '+str(precision))
    print('Recall: '+str(recall))
    print('Accuracy: '+str((tp + tn) / (tp + fp + tn + fn)))
    print('     T       F')
    print('P    '+str(tp)+' '*(8-len(str(tp)))+str(fp))
    print('N    '+str(tn)+' '*(8-len(str(tn)))+str(fn))

def k_fold_gap_validation(get_model, data, annotations, k, lower, upper, step):
    N = len(data)
    frames_predictions = []
    scores = []
    for i in range(k):
        model = get_model()
        model.fit(np.concatenate((data[:i * N // k], data[(i + 1) * N // k:])), np.concatenate((
            annotations[:i * N // k], annotations[(i + 1) * N // k:])))
        frames_predictions.append(model.predict(data[i * N // k: (i + 1) * N // k]))   
    for gap in range(lower, upper, step):
        score = 0
        for i in range(k):
            frames_predicted = frames_predictions[i]
            frames_smoothed = smooth(frames_predicted)
            frames_merged = merge(frames_smoothed, gap) 
            score += map2014(frames_to_intervals(frames_merged), frames_to_intervals(
                annotations[i * N // k: (i + 1) * N // k]))
        scores.append(score / k)
    best = max(scores)
    return scores.index(best) * step + lower, best

In [ ]:
def save_model(save_path, model, model_name):
    os.chdir(save_path + r'/models')
    model.save(model_name + '.h5')
    
def load_model(save_path, model_name):
    os.chdir(save_path + r'/models')
    return models.load_model(model_name + '.h5')

def get_layer_output(model, layer_name, data): # use for neural networks
    extractor = Model(inputs=model.inputs, outputs=[model.get_layer(layer_name).output])
    res = extractor.predict(data)
    res = np.array(res)
    return res 

def multi_loss_3(y_true, y_pred):
    loss = 0
    weights = [1 / 3] * 3
    bce = bce = tf.keras.losses.binary_crossentropy
    for i in range(3):
        s = 0
        s += bce(y_true[:,i], y_pred[:,i])
        loss += weights[i] * s
    return loss 

def get_sequence(data, window): # use for LSTM
    sequence = []
    for i in range(0, data.shape[0] - window, window):
        sequence.append(data[i: i + window])
    i += window
    if i < data.shape[0]:
        if window - data.shape[0] + i == 0:
            sequence.append(data[i: data.shape[0]])
        else:
            sequence.append(np.concatenate((data[i: data.shape[0]], [data[-1]] * (window - data.shape[0] + i))))
    return np.array(sequence)   

def smooth(frames, window = 25, threshold = 0.5): # 25 fps
    smoothed_frames = []
    frames_number = len(frames)
    i = 0
    while i < frames_number:
        window_score = 0
        j = 0
        while i + j < frames_number and j < window:
            window_score += frames[i]
            i += 1
            j += 1
        if window_score / j >= threshold:
            smoothed_frames.extend([1] * j)
        else:
            smoothed_frames.extend([0] * j)
    return np.array(smoothed_frames)

def merge(frames, gap):
    frames_number = len(frames)
    new_frames = frames.copy()
    i = 0
    while i < frames_number:
        if new_frames[i] == 1:
            while i < frames_number and new_frames[i] == 1:
                i += 1
            j = i
            merge_flag = False
            while j < min([frames_number, i + gap]):
                if new_frames[j] == 1:
                    merge_flag = True
                    break
                j += 1
            if merge_flag:
                while i < j:
                    new_frames[i] = 1
                    i += 1
        else:
            i += 1
    return np.array(new_frames)

def compress(frames, window, threshold = 0.5):
    frames_compressed = []
    for i in range(0, frames.shape[0] - window, window):
        if sum(frames[i: i + window]) / window >= threshold:
            frames_compressed.append(1)
        else:        
            frames_compressed.append(0)
    i += window
    if i < frames.shape[0]:
        if (sum(frames[i: frames.shape[0]]) + frames[-1] * (window - frames.shape[0] + i)) / window >= threshold:
            frames_compressed.append(1)
        else:
            frames_compressed.append(0)
    return np.array(frames_compressed) 
    
def decompress(frames, window, length = None):
    frames_decompressed = []
    for a in frames:
        frames_decompressed.extend([a] * window)
    if length != None:
        frames_decompressed = frames_decompressed[:length]
    return np.array(frames_decompressed)  
    
def frames_to_intervals(frames):
    intervals = []
    frames_number = len(frames)
    i = 0
    while i < frames_number:
        if frames[i] == 1:
            j = i
            while j < frames_number and frames[j] == 1:
                j += 1    
            intervals.append((i, j))
            i = j
        else:
            i += 1
    return intervals

def get_predictions(sigmoid_values):
    y_pred = []
    l = sigmoid_values.shape[0]
    for i in range(l):
        y_pred.append(round(sigmoid_values[i][0]))
    return np.array(y_pred)

def class_weight(frames, fp_bias = 0):
    positives = np.sum(frames)
    negatives = np.size(frames) - positives
    return {0 : 1 + positives/negatives, 1: 1 + (negatives + fp_bias)/positives}

def one_hot_encode(labelled_frames):
    labels_dictionary = {}
    label = 0
    for l in set(labelled_frames):
        labels_dictionary[l] = label
        label += 1
    res = []
    for l in labelled_frames:
        encoding = np.array([0] * label)
        encoding[labels_dictionary[l]] = 1
        res.append(encoding)  
    return np.array(res)                   
    
def azip(ys):
    res = []
    for i in range(ys[0].shape[0]):
        column = []
        for j in range(len(ys)):
                for k in range(ys[j].shape[1]):
                    column.append(ys[j][i][k])
        res.append(np.array(column))
    return np.array(res)  

In [ ]:
def evaluate(model, data, frames_test, gap = None): 
    frames_predicted = np.array(model.predict(data))
    print_conf_matrix(frames_predicted, frames_test)
    print(map_classic(frames_to_intervals(frames_predicted), frames_to_intervals(frames_test)))
    frames_smoothed = smooth(frames_predicted)
    print(map_classic(frames_to_intervals(frames_smoothed), frames_to_intervals(frames_test)))
    best_score = 0
    best_gap = 0
    if gap != None:
        best_gap = gap
    else:
        for gap in range(0, 600, 100): 
            frames_merged = merge(frames_smoothed, gap)
            score = map_classic(frames_to_intervals(frames_merged), frames_to_intervals(frames_test))
            print((gap, score), end = '  ')
            if score > best_score:
                best_score = score
                best_gap = gap
    print()
    print(str(best_score) + ' ' + str(best_gap))
    frames_merged = merge(frames_smoothed, best_gap)
    print_conf_matrix(frames_merged, frames_test)
    return best_score